In [3]:

# image files. There are 43 unique classes in total. The images are of shape
# (30,30,3).
# ==============================================================================

# HINT: Your neural network must have a validation accuracy of approximately
# 0.95 or above on the normalized validation dataset for top marks.
 

In [7]:
import urllib
import zipfile
 
import tensorflow as tf
 
# This function downloads and extracts the dataset to the directory that
# contains this file.
# DO NOT CHANGE THIS CODE
# (unless you need to change https to http)


In [9]:
import wget
import pathlib
def download_and_extract_data():
    file1 = pathlib.Path("germantrafficsigns.zip")
    file2 = pathlib.Path("germantrafficsigns.zip")
    if file2.exists ():
        print("") #("CSV file exist")
    elif file1.exists():
        print("Zip file exist")
        with zipfile.ZipFile('germantrafficsigns.zip', 'r') as zip_ref:
            zip_ref.extractall()    
    else:
        wget.download('https://storage.googleapis.com/download.tensorflow.org/data/certificate/germantrafficsigns.zip')
        with zipfile.ZipFile('germantrafficsigns.zip', 'r') as zip_ref:
            zip_ref.extractall()
down_file = download_and_extract_data() 

In [10]:
 # COMPLETE THE CODE IN THIS FUNCTION
def preprocess(image, label):
   # NORMALIZE YOUR IMAGES HERE (HINT: Rescale by 1/.255)
   image = image /255
   labelZip = label
   return image, labelZip
 
 
# This function loads the data, normalizes and resizes the images, splits it into
# train and validation sets, defines the model, compiles it and finally
# trains the model. The trained model is returned from this function.
import os
base_dir = "./"
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

train_fname = os.listdir(train_dir)
val_fname = os.listdir(val_dir)
print(len(train_fname))
print(len(val_fname))
print(train_fname[:10])



43
43
['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009']


In [11]:

 
# COMPLETE THE CODE IN THIS FUNCTION.
def solution_model(train_dir, val_dir):
   # Downloads and extracts the dataset to the directory that
   # contains this file.
      
   # download_and_extract_data()
 
   BATCH_SIZE = 32
   IMG_SIZE = 30
 
   # The following code reads the training and validation data from their
   # respective directories, resizes them into the specified image size
   # and splits them into batches. You must fill in the image_size
   # argument for both training and validation data.
   # HINT: Image size is a tuple
   train_ds = tf.keras.preprocessing.image_dataset_from_directory(
       directory=train_dir,
       label_mode='categorical',
       image_size=  (30, 30),
       batch_size = BATCH_SIZE)
 
   val_ds = tf.keras.preprocessing.image_dataset_from_directory(
       directory=val_dir,
       label_mode='categorical',
       image_size= (30, 30),
       batch_size = BATCH_SIZE,)
   
   return train_ds, val_ds
 


In [12]:
train_ds, val_ds = solution_model(train_dir, val_dir)

Found 31367 files belonging to 43 classes.
Found 7842 files belonging to 43 classes.


In [13]:

   # Normalizes train and validation datasets using the
   # preprocess() function.
   # Also makes other calls, as evident from the code, to prepare them for
   # training.
   # Do not batch or resize the images in the dataset here since it's already
   # been done previously.
def img_process(train_ds, val_ds):
        train_img = train_ds.map(
            preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(
            tf.data.experimental.AUTOTUNE)
        val_img = val_ds.map(
            preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        
        return train_img, val_img

train_img, val_img = img_process(train_ds, val_ds)
 

In [16]:
# Code to define the model
model = tf.keras.models.Sequential([
    # ADD LAYERS OF THE MODEL HERE

    # If you don't adhere to the instructions in the following comments,
    # tests will fail to grade your model:
    # The input layer of your model must have an input shape of
    # (30,30,3).
    # Make sure your last layer has 43 neurons activated by softmax.
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(30, 30, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(43, activation='softmax')
])

print(model.summary())

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    # Check accuracy
    if(logs.get('accuracy') > 0.95):

      # Stop if threshold is met
      print("\nLoss is more than 0.95 so cancelling training!")
      self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 64)        1792      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               1180160   
_________________________________________________________________
dense_5 (Dense)              (None, 43)               

In [17]:
# Code to compile and train the model
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


model.fit(train_img,
          steps_per_epoch=8,  
          epochs=100,
          verbose=1,
          validation_data = val_img,
          validation_steps=8,
          callbacks=[callbacks])
        # YOUR CODE HERE


 

Epoch 1/100
8/8 [==============================] - 6s 536ms/step - loss: 3.7072 - accuracy: 0.0547 - val_loss: 3.5988 - val_accuracy: 0.0742
Epoch 2/100
8/8 [==============================] - 1s 197ms/step - loss: 3.5734 - accuracy: 0.0664 - val_loss: 3.5541 - val_accuracy: 0.1172
Epoch 3/100
8/8 [==============================] - 1s 183ms/step - loss: 3.4856 - accuracy: 0.1133 - val_loss: 3.4644 - val_accuracy: 0.0977
Epoch 4/100
8/8 [==============================] - 1s 190ms/step - loss: 3.3678 - accuracy: 0.1172 - val_loss: 3.3609 - val_accuracy: 0.1094
Epoch 5/100
8/8 [==============================] - 1s 178ms/step - loss: 3.2383 - accuracy: 0.1562 - val_loss: 3.3345 - val_accuracy: 0.1406
Epoch 6/100
8/8 [==============================] - 2s 215ms/step - loss: 3.2502 - accuracy: 0.1328 - val_loss: 3.1522 - val_accuracy: 0.1680
Epoch 7/100
8/8 [==============================] - 2s 220ms/step - loss: 3.0006 - accuracy: 0.1602 - val_loss: 2.9706 - val_accuracy: 0.2578
Epoch 8/100
8

In [ ]:
# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.
'''if __name__ == '__main__':
   model = solution_model()
   model.save("mymodel.h5")
'''

In [ ]:
model.save("mymodelq3.h5")